In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.externals import joblib
from cv2 import HOGDescriptor

def compute_features(image):
    winSize = (16,48)
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 100
    nlevels = 32
    hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    winStride = (8,8)
    padding = (16,16)
    hist = hog.compute(image,winStride,padding)
    return np.reshape(hist, 4500)

clf = joblib.load('model_semafors.pkl')

cap = cv2.VideoCapture('test2.mp4')
frame = cap.read()[1]
frame = cv2.resize(frame,(640, 480), interpolation = cv2.INTER_CUBIC)
height, width = frame.shape[:2]
while(cap.isOpened()):
    frame=cap.read()[1]
    frame = cv2.resize(frame,(640, 360), interpolation = cv2.INTER_CUBIC)
    low_red = (0,0,200)
    hight_red = (140,140,255)
    mask = cv2.inRange(frame, low_red, hight_red)
    mask = cv2.Canny(mask,20,220)    
    mask = cv2.dilate(mask, (3,3), iterations=1)
    cont = cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)[1] 
    for cnt in cont:
        L = tuple(cnt[cnt[:, :, 0].argmin()][0])
        R = tuple(cnt[cnt[:, :, 0].argmax()][0])
        T = tuple(cnt[cnt[:, :, 1].argmin()][0])
        B = tuple(cnt[cnt[:, :, 1].argmax()][0])
        area = abs(cv2.contourArea(cnt))
        perim = cv2.arcLength(cnt, True)
        if(perim!=0):
            if(perim>27 and cv2.matchShapes(cnt,cont[1],cv2.CONTOURS_MATCH_I3,0.0)<0.03):
                diam = R[0] - L[0]
                cv2.drawContours(frame,cnt,-1, (0, 255, 0), 1)
                cut_frame = frame[(B[1]-diam):(T[1]+diam*4),(B[0]-diam):(T[0]+diam)]
                if np.all(cut_frame) != None:
                    cut_frame = cv2.resize(cut_frame,(16, 48), interpolation = cv2.INTER_CUBIC)
                    pred = clf.predict_proba([compute_features(cut_frame)])[0][1]
                    print(pred)
                    if(pred>0.5):
                        cv2.rectangle(frame,(B[0]-diam,B[1]-diam),(T[0]+diam,T[1]+diam*4),(0,0,255),2)
    cv2.imshow('frame2',mask)
    cv2.imshow('frame3',frame)
    if cv2.waitKey(2) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()